# 自転車犯罪マップ

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/crime map.png" width=600>

このラボの目的:

- オープンデータソースからデータを取得し、Python ノートブックにインポート
- 複数の列から単一の住所フィールドを作成
- 緯度と経度の座標を割り当てて各行をジオコーディング
- 美しいチャートや地図を作成

## ライブラリをインポートする

このラボで使うライブラリーを一気にインポートしよう。

In [1]:
## for spatial analysis
import geopandas as gpd

## for data analysis
import pandas as pd

## for pretty charts
import plotly.express as px

# for plotly themes テーマ
import plotly.io as pio

## for URL requests
import urllib.request
import requests

## for maps
import folium
from folium import plugins

## 自転車盗難データをダウンロード

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/Chiba%20police.png" width=400>

まずは千葉県警察のサイトにアクセス。そこから次に手順でデータをダウンロード：

➡️ https://www.police.pref.chiba.jp/

➡️ 安全な暮らし

➡️ 地域の防犯

➡️ あなたの町の犯罪情勢

➡️ オープンデータ

➡️ 自転車盗（CSV）

ダウンロードしたファイルを `chibike.csv` と名付けて、このフォルダー（Week08) にセーブ。

In [7]:
# ダウンロードしたデータを読み込もう
df = pd.read_csv('chibike.csv', encoding='cp932')

In [6]:
# データの情報
#objuct　質的データ
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7346 entries, 0 to 7345
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             7346 non-null   object
 1   手口             7346 non-null   object
 2   管轄警察署（発生地）     7346 non-null   object
 3   管轄交番・駐在所（発生地）  7343 non-null   object
 4   市区町村コード（発生地）   7346 non-null   int64 
 5   都道府県（発生地）      7346 non-null   object
 6   市区町村（発生地）      7346 non-null   object
 7   町丁目（発生地）       7330 non-null   object
 8   発生年月日（始期）      7346 non-null   object
 9   発生時（始期）        7346 non-null   object
 10  発生場所           7346 non-null   object
 11  発生場所の詳細        7346 non-null   object
 12  被害者の年齢         7346 non-null   object
 13  被害者の職業         7346 non-null   object
 14  施錠関係           7346 non-null   object
dtypes: int64(1), object(14)
memory usage: 861.0+ KB


Wow! ７千件もある！ちょっと多いので場所で絞りましょう。<h1>「柏」</h1>だけのデータフレームを作ろう。

In [4]:
kashiwa = df[df['管轄警察署（発生地）'] == '柏'].copy()

In [5]:
# データをチェック
kashiwa.sample(5)

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,被害者の年齢,被害者の職業,施錠関係
2851,窃盗,自転車盗,柏,南増尾交番,122173,千葉県,柏市,南増尾１丁目,2022-12-18,19,その他,駐車（輪）場,40歳代,その他,施錠した
2919,窃盗,自転車盗,柏,増尾駅前交番,122173,千葉県,柏市,増尾１丁目,2022-07-21,17,道路上,その他,30歳代,その他,施錠せず
3195,窃盗,自転車盗,柏,花野井交番,122173,千葉県,柏市,大室,2022-02-02,19,その他の住宅（３階建て以下共同住宅等）,駐車（輪）場,10歳代,高校生,施錠した
2890,窃盗,自転車盗,柏,南柏駅前交番,122173,千葉県,柏市,南柏１丁目,2022-11-08,19,その他,駐車（輪）場,50歳代,その他,施錠せず
3065,窃盗,自転車盗,柏,柏の葉キャンパス交番,122173,千葉県,柏市,若柴,2022-04-27,10,その他,駐車（輪）場,20歳代,その他,施錠した


### `value_counts`でチャート

このデータを見て、咄嗟に知りたいものってなんでしょう？

例えば、<h1>「1日の何時に自転車の盗難が一番発生するの？」</h1>の質問に答えるためにはどのようなデータ分析が必要でしょうか？

では、そのチャートを作ってみましょう。

データには **【発生時（始期）】** のカラムがあるので、各時間帯のカウントを`value_counts()`で調べる。最後に足される`reset_index()`で結果をデータフレームに変換します。

In [8]:
# create a new variable with hourly counts
time = kashiwa['発生時（始期）'].value_counts().reset_index()
time

,発生時（始期）,count
0,18,44
1,19,39
2,08,37
3,16,36
4,20,33
5,17,32
6,12,28
7,13,25
8,15,24
9,07,23


この結果は良いが、カラム名（ヘッダー）を直す必要がある。

In [9]:
# fix headers
time.columns = ['発生時（始期）','件数']
time

,発生時（始期）,件数
0,18,44
1,19,39
2,08,37
3,16,36
4,20,33
5,17,32
6,12,28
7,13,25
8,15,24
9,07,23


いよいよ準備万端。この新しいデータフレームでチャートを作ろう。この場合は plotly express の bar charts を参考。

https://plotly.com/python/bar-charts/

In [10]:
fig = px.bar(time,x='発生時（始期）',y='件数')
fig.show()

ありゃ？順番が件数の多い順になってる。実はx軸のオプションっていっぱいあるんだ！その中の一つでカテゴリーの順番を設定できる。

https://plotly.com/python/categorical-axes/

In [11]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数'
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

チャートのテンプレートを変えることでルックスが変わる。

`template`のオプションはこちらから：

```["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white"]```

一つずつ試してみよう。

In [12]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数',
            template='ggplot2' # change this to see other styles
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

### Make your own charts

では、ここで他のチャートを作ってみよう。

例：被害者の年齢、被害者の職業

In [16]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数',
            template='plotly_white' # change this to see other styles
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [17]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数',
            template='simple_white' # change this to see other styles
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [19]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数',
            template='plotly' # change this to see other styles
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

## その他のチャート（上級編）

複数の変数でチャートを作ると、より深い分析ができる。

In [21]:
fig = px.bar(kashiwa,
            x='発生時（始期）',
            color='被害者の職業',
            template='seaborn')
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

↑上のチャートの凡例ののカテゴリーをダブルクリックするとどうなる？

`barmode='group'`を足すと何がどのように変わる？

In [22]:
fig = px.bar(kashiwa,
            x='発生時（始期）',
            color='被害者の職業',
            barmode='group', # group the categories,
            template='seaborn'
            )
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

`facet_col=`を足すと複数のチャートを一気に表示できる。

In [23]:
fig = px.bar(kashiwa,
            x='発生時（始期）',
            facet_col='被害者の職業',
            template='seaborn')
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

# Geocoding

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/Geocoding_01.png" width=400>

住所だけではマップイングできません。座標が必要です。なので、住所から座標を特定するプロセスが必要である。このプロセスを<h1>【ジオコーティング】</h1>という。


ジオコーティングと言えば、色んな方法があります。現在、日本で無料でジオコーティングサービスを提供しているのが国土地理院のジオコーティングAPI。

試してみよう。このようにURLをブラウザーで記入するだけで座標が返ってくるサービスである。国土地理院さん、とても便利なサービス、ありがとうございます！

https://msearch.gsi.go.jp/address-search/AddressSearch?q=麗澤大学

では、このプロセスに従って、アドレスから座標を出力する Python 関数を作成します。

In [28]:
# 関数を作成
def geocode(address):

    # ジオコーティングURL
    url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

    # try/exceptでエラーをキャッチ
    # 成功の場合
    try:
        s_quote = urllib.parse.quote(address)
        response = requests.get(url + s_quote)
        if len(response.json())>0:
            return response.json()[0]["geometry"]["coordinates"] 
        else:
            return False
        
    # 失敗の場合
    except:
        return False

関数の使い方は簡単！

In [34]:
geocode('旭川動物園')

[142.365005, 43.770832]

## データフレームの準備

`kashiwa`のデータフレームの各行の住所をジオコーティングする前に`住所`と座標（`lat`,`lon`）のフィールドを作りましょう。

In [35]:
# 空の住所フィールドを作成
kashiwa['住所'] = ''

In [36]:
# 空だがfloatとしてフィールドを作成
kashiwa['lat'] = pd.Series(dtype=float)
kashiwa['lon'] = pd.Series(dtype=float)

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/concat.png" width=600>

↑で作った住所フィールドに次の 3 つのフィールドを連結します。

1. 都道府県（発生地）
1. 市区町村（発生地）
1. 町丁目（発生地）

In [37]:
# 住所フィールドを作成
kashiwa['住所'] = kashiwa['都道府県（発生地）']+kashiwa['市区町村（発生地）']+kashiwa['町丁目（発生地）']

In [38]:
# random sampleで上手く行ったかどうかをチェック
kashiwa.sample(3)

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,被害者の年齢,被害者の職業,施錠関係,住所,lat,lon
3006,窃盗,自転車盗,柏,旭町交番,122173,千葉県,柏市,末広町,2022-08-02,14,その他,駐車（輪）場,10歳代,その他,施錠せず,千葉県柏市末広町,NaN,NaN
2798,窃盗,自転車盗,柏,北柏駅前交番,122173,千葉県,柏市,柏下,2022-10-31,09,その他,その他,65-69歳,その他,施錠せず,千葉県柏市柏下,NaN,NaN
3029,窃盗,自転車盗,柏,松葉町交番,122173,千葉県,柏市,根戸,2022-12-10,12,その他,駐車（輪）場,20歳代,その他,施錠した,千葉県柏市根戸,NaN,NaN


これで準備は整えました。`kashiwa` のデータフレームを for loop に入れて住所を一つずつジオコーティングしよう。

この作業は数分かかるので注意。千件以上のデータフレームはなるべく避けよう。

In [39]:
# kashiwaデータフレームをループ
for i,row in kashiwa.iterrows():

    # ジオコーティング成功
    if geocode(row['住所']) != False:
        # 座標を変数に
        lon = geocode(row['住所'])[0]
        lat = geocode(row['住所'])[1]

        # データフレームに値をインプット        
        kashiwa.loc[i,'lon'] = lon
        kashiwa.loc[i,'lat'] = lat

        # 結果をprint
        print(row['住所'],lon,lat)
    
    # ジオコーティング失敗
    else:
        print(row['住所'],'ジオコーティング失敗')
        continue

千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市酒井根 139.957825 35.81815
千葉県柏市酒井根３丁目 139.965973 35.818859
千葉県柏市酒井根４丁目 139.960007 35.820488
千葉県柏市酒井根５丁目 139.961349 35.824585
千葉県柏市酒井根５丁目 139.961349 35.824585
千葉県柏市逆井 139.982468 35.815441
千葉県柏市中新宿３丁目 139.946884 35.83292
千葉県柏市西山２丁目 139.95401 35.82056
千葉県柏市光ケ丘１丁目 139.957794 35.833195
千葉県柏市光ケ丘１丁目 139.957794 35.833195
千葉県柏市光ケ丘団地 139.957809 35.829479
千葉県柏市東中新宿１丁目 139.960556 35.833984
千葉県柏市東中新宿１丁目 139.960556 35.833984
千葉県柏市東中新宿１丁目 139.960556 35.833984
千葉県柏市柏 139.986511 35.864445
千葉県柏市柏 139.986511 35.864445
千葉県柏市柏 139.986511 35.864445
千葉県柏市柏下 139.983795 35.870007
千葉県柏市柏堀之内新田 139.983582 35.875332
千葉県柏市北柏２丁目 139.989914 35.872643
千葉県柏市北柏２丁目 139.989914 35.872643
千葉県柏市北柏２丁目 139.989914 35.872643
千葉県柏市北柏３丁目 139.987793 35.872192
千葉県柏市北柏３丁目 139.987793 35.872192
千葉県柏市北柏４丁目 139.986893 35.870586
千葉県柏市北柏４丁目 139.986893 35.870586
千葉県柏市北柏５丁目 139.990295 3

In [40]:
# check your output
kashiwa.sample(5)

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,被害者の年齢,被害者の職業,施錠関係,住所,lat,lon
3117,窃盗,自転車盗,柏,柏駅前交番,122173,千葉県,柏市,柏３丁目,2022-04-25,18,駐車（輪）場,駐車（輪）場,20歳代,その他,施錠せず,千葉県柏市柏３丁目,35.860302,139.975159
2925,窃盗,自転車盗,柏,増尾駅前交番,122173,千葉県,柏市,増尾台４丁目,2022-09-05,21,一戸建住宅,その他,10歳代,高校生,施錠せず,千葉県柏市増尾台４丁目,35.821732,139.973206
3185,窃盗,自転車盗,柏,緑ヶ丘交番,122173,千葉県,柏市,若葉町,2022-03-18,20,４階建て以上共同住宅,駐車（輪）場,10歳代,高校生,施錠せず,千葉県柏市若葉町,35.853920,139.971054
3042,窃盗,自転車盗,柏,松葉町交番,122173,千葉県,柏市,松葉町４丁目,2022-04-22,18,４階建て以上共同住宅,その他,70歳以上,その他,施錠した,千葉県柏市松葉町４丁目,35.889114,139.971802
3057,窃盗,自転車盗,柏,松葉町交番,122173,千葉県,柏市,大山台２丁目,2022-12-20,16,その他,駐車（輪）場,30歳代,その他,施錠せず,千葉県柏市大山台２丁目,35.882580,139.969193


## Let's map it!

待ってました、いよいよマップタイム！

In [41]:
# 全データの中央座標
center_lat = kashiwa.lat.mean()
center_lon = kashiwa.lon.mean()

In [42]:
# make the map
m = folium.Map(location=[center_lat,center_lon], 
               zoom_start=12)

# マーカーがメチャクチャ多い場合はclusterで処理！
marker_cluster = plugins.MarkerCluster().add_to(m)
 
# kashiwaのデータフレームをループしてマーカーを作る
for index, row in kashiwa.iterrows():
    latlon = [row['lat'],row['lon']]
    folium.Marker(latlon, 
                  tooltip=row['被害者の年齢'],
                ).add_to(marker_cluster) # mapにではなくmarker_clusterに足す

# show the map
m

# EXTRA Time?

時間が余った人は違う場所の犯罪マップ作りに挑戦。

このサイトから都道府県を選び、データをダウンロード：

https://www.npa.go.jp/toukei/seianki/hanzaiopendatalink.html

注意！ジオコーティングは時間がかかるので、データを千件以下に絞ってからジオコーティングをするように。